# Final Project

제공된 이미지(혹은 자신이 원하는 이미지)에 대해

1. 필터를 적용한 결과물을 원본과 함께 나열하여 하나의 이미지 파일로 저장합니다.
2. 1의 작업을 3개 이상의 필터에 대해 반복 수행합니다.
3. `이미지이름_필터이름.xlsx`에 2의 결과물을 각각 저장합니다.
4. 3의 결과 생성된 여러 엑셀 파일을 하나의 파일 `이미지이름_filters.xlsx`로 병합합니다.

## Input

- 제공된(혹은 www.pexels.com 에서 각자 준비한) 이미지

## Expected Output

- 이미지이름_filters.xlsx

## 주의사항

- 아래의 절차를 따르지 않으셔도 좋지만, 따르지 않았을 때 평가기준을 벗어날 위험이 있습니다.
- 이미지를 주피터 노트북에서 바로 로드하여 저장하지 마세요(파일 크기가 증가하여 업로드에 실패하고 노트북 로드가 잘 되지 않을 수 있습니다.)
- 결과물을 압축하여 업로드하세요.
- 노트북이 잘 저장되었는지 꼭 확인하세요(`ctrl+s`)

0. 필요한 라이브러리는 아래 블록에 일괄적으로 import 해주세요

In [170]:
# Library import block
from PIL import Image, ImageFilter
import xlwings as xw
import os
import glob

1. 이미지를 불러오세요

In [171]:
from PIL import Image, ImageFilter
img = Image.open('./house.jpg')

2. 불러온 이미지에 대해 필터 작업을 수행하세요

In [172]:
img_filter_1 =  img.filter(ImageFilter.GaussianBlur(15))

3. 원본과 필터를 적용한 결과물을 담을 캔버스를 생성한 뒤, 두 이미지를 캔버스에 나란히 나열하세요

In [173]:
img.size, img_filter_1.size

((4096, 2726), (4096, 2726))

In [174]:
x, y = img.size

In [175]:
blk_x = int((9000-2*x)/3)
blk_y = int((3000-y)/2)
blk_x, blk_y

(269, 137)

In [176]:
bg_img = Image.new("RGB", (9000,3000), '#dcfcfa')
bg_img.paste(img, (blk_x, blk_y))
bg_img.paste(img_filter_1, (x+2*blk_x, blk_y))

4. 이미지를 저장하세요

In [177]:
bg_img.save('./house_with_blur.jpg')

5. 3개 이상의 필터에 대해 1~4의 작업을 반복수행하세요(본인이 편한 방법으로 반복)

In [178]:
various_filters = {
    'contour' : ImageFilter.CONTOUR(),
    'emboss' : ImageFilter.EMBOSS(),
    'detail' : ImageFilter.DETAIL()
}


In [179]:
for filter_name, filter_type in various_filters.items():
    filter_img =  img.filter(filter_type)
    bg_img = Image.new("RGB", (9000,3000), '#dcfcfa')
    bg_img.paste(img, (blk_x, blk_y))
    bg_img.paste(filter_img, (x+2*blk_x, blk_y))
    bg_img.save(f'./house_with_{filter_name}.jpg')

6. xlwings를 활용하여 새로운 Workbook을 생성하세요

7. 원하는 위치에 저장한 이미지를 새로운 시트에 붙여넣으세요(시트이름: `필터이름`)

8. 결과물을 확인하고 Workbook을 5의 결과물 이미지들에 대해 독립된 xlsx 파일로 저장하세요(파일명: `이미지이름_필터이름.xlsx`)

In [180]:
imgs = {
    'contour' : r'C:\Users\user\Desktop\project\house_with_contour.jpg',
    'emboss' : r'C:\Users\user\Desktop\project\house_with_emboss.jpg',
    'detail' : r'C:\Users\user\Desktop\project\house_with_detail.jpg',
}

In [181]:
for name, file in imgs.items():
    wb = xw.Book()
    ws = wb.sheets.add(f'{name}')
    ws.pictures.add(
            file,
            top=ws.range('B2').top,
            left=ws.range('B2').left,
            scale=0.1,
            )
    wb.save(f'./house_{name}.xlsx')
    wb.close()
        

9. 8의 결과물들의 워크시트들을 하나의 문서(파일명: `이미지이름_filters.xlsx`)에 병합하여 저장하세요

In [183]:
to_merge_list = glob.glob(f'./*.xlsx')

In [185]:
merge_target = xw.Book()

In [186]:
for file in to_merge_list:
    wb = xw.Book(file)
    ws = wb.sheets[0]
    ws.copy(before=merge_target.sheets['Sheet1'])
    wb.close()
merge_target.sheets['Sheet1'].delete()
merge_target.save('./house_filters.xlsx')
merge_target.close()